In [1]:
from sklearn.metrics import roc_auc_score

In [2]:
import pickle

In [3]:
import pandas as pd

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
data = pd.read_csv('data_sample_500.csv')
data

,terms,definitions,source,assigned_readability,flesch_reading_ease,flesch_kincaid_grade,smog_index,coleman_liau_index,automated_readability_index,dale_chall_readability_score,difficult_words,linsear_write_formula,gunning_fog,text_standard,fernandez_huerta,szigriszt_pazos,gutierrez_polini,crawford,gulpease_index,osman
0,NEGATIVE CARRY,Any transaction or TRADE where the RETURNS gen...,palgrave,0,56.25,9.1,0.0,9.21,8.8,9.64,9,8.000000,10.00,9th and 10th grade,95.6,94.23,44.68,2.9,60.666667,68.69
1,Consumer Discretionary,Consumer discretionary is a term for classifyi...,investopedia,1,24.27,15.2,0.0,20.07,20.6,12.41,18,16.250000,18.83,20th and 21st grade,70.9,69.43,25.19,4.8,36.441860,9.70
2,OBJECTIVE RISK,A measure of the deviation between an INSURER’...,palgrave,0,29.35,13.3,0.0,17.75,16.2,11.63,13,13.250000,18.72,12th and 13th grade,76.0,69.51,30.34,4.9,44.757576,23.57
3,Patents,In 1899 the commissioner of the American Offic...,economist,1,54.05,12.1,14.4,11.20,14.5,8.85,35,16.250000,14.32,14th and 15th grade,90.8,86.97,39.15,3.2,51.549020,49.50
4,MASTER FUND,A FUND that invests CAPITAL gathered from INVE...,palgrave,0,75.20,6.0,0.0,10.60,8.8,10.96,8,6.000000,5.20,5th and 6th grade,109.6,105.18,42.52,2.0,61.692308,59.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,FRIENDLY TAKEOVER,An ACQUISITION that is agreed on amicable term...,palgrave,0,48.64,12.1,0.0,10.86,12.8,9.31,9,14.750000,13.44,12th and 13th grade,87.9,84.66,40.77,3.5,53.666667,53.29
496,Continuous Variable,A quantitative variable that can take any nume...,ncert_kest,1,36.96,10.3,0.0,12.50,9.3,9.35,3,5.500000,12.49,9th and 10th grade,83.7,80.16,39.24,3.6,66.777778,57.32
497,revolving credit,legally assured line of credit with a bank.,prin,0,80.28,4.1,0.0,5.76,3.8,9.95,3,3.000000,3.20,3rd and 4th grade,114.7,113.17,49.96,0.8,81.500000,83.86
498,Trust Fund,A trust fund is an estate planning tool that e...,investopedia,1,52.19,10.7,13.0,9.52,10.5,9.77,14,12.500000,12.34,9th and 10th grade,91.5,90.29,43.75,3.2,57.157895,61.87


In [6]:
data['assigned_readability'].value_counts(normalize = True)

1    0.58
0    0.42
Name: assigned_readability, dtype: float64

In [7]:
data['assigned_readability'].value_counts()

1    290
0    210
Name: assigned_readability, dtype: int64

In [8]:
data[data['assigned_readability']==1]['difficult_words'].mean()

13.610344827586207

In [9]:
data[data['assigned_readability']==1]['difficult_words'].median()

14.0

In [10]:
data[data['assigned_readability']==1]['difficult_words'].mode()

0    14
dtype: int64

In [11]:
data[data['assigned_readability']==0]['difficult_words'].mean()

7.819047619047619

In [12]:
data[data['assigned_readability']==0]['difficult_words'].median()

6.0

In [13]:
data[data['assigned_readability']==0]['difficult_words'].mode()

0    1
dtype: int64

In [14]:
data[['source', 'assigned_readability']].value_counts().reset_index()

,source,assigned_readability,0
0,investopedia,1,227
1,palgrave,0,155
2,zvi,0,18
3,economist,1,18
4,6_8_louis,1,17
5,fmi,0,16
6,opod,0,15
7,sam,1,11
8,prin,0,6
9,9_12_louis,1,6


In [15]:
data.columns

Index(['terms', 'definitions', 'source', 'assigned_readability',
       'flesch_reading_ease', 'flesch_kincaid_grade', 'smog_index',
       'coleman_liau_index', 'automated_readability_index',
       'dale_chall_readability_score', 'difficult_words',
       'linsear_write_formula', 'gunning_fog', 'text_standard',
       'fernandez_huerta', 'szigriszt_pazos', 'gutierrez_polini', 'crawford',
       'gulpease_index', 'osman'],
      dtype='object')

In [16]:
numeric_columns = ['assigned_readability',
       'flesch_reading_ease', 'flesch_kincaid_grade', 'smog_index',
       'coleman_liau_index', 'automated_readability_index',
       'dale_chall_readability_score', 'difficult_words',
       'linsear_write_formula', 'gunning_fog',
       'fernandez_huerta', 'szigriszt_pazos', 'gutierrez_polini', 'crawford',
       'gulpease_index', 'osman']

In [17]:
for col in numeric_columns:
  auroc = roc_auc_score(data['assigned_readability'].values, data[col].values)
  print(col,auroc,"\n")

assigned_readability 1.0 

flesch_reading_ease 0.4605665024630542 

flesch_kincaid_grade 0.6283743842364533 

smog_index 0.6477586206896552 

coleman_liau_index 0.56376026272578 

automated_readability_index 0.6388341543513958 

dale_chall_readability_score 0.4288259441707718 

difficult_words 0.7440804597701149 

linsear_write_formula 0.6893349753694581 

gunning_fog 0.5626600985221675 

fernandez_huerta 0.4340886699507389 

szigriszt_pazos 0.4396059113300492 

gutierrez_polini 0.44817733990147784 

crawford 0.6091789819376028 

gulpease_index 0.314055829228243 

osman 0.40876026272578 



In [18]:
features = ['definitions']
Y = data['assigned_readability']

In [19]:
import numpy as np
np.random.seed(0)
msk = np.random.rand(len(data)) < 0.8
df = data[msk]
df = df.reset_index().copy()
val_df = data[~msk]
val_df = val_df.reset_index().copy()

In [20]:
for col in numeric_columns:
  auroc = roc_auc_score(val_df['assigned_readability'].values, val_df[col].values)
  print(col,auroc,"\n")

assigned_readability 1.0 

flesch_reading_ease 0.49304650152107776 

flesch_kincaid_grade 0.5895262929161234 

smog_index 0.6134289439374185 

coleman_liau_index 0.5780095610604085 

automated_readability_index 0.646458061712299 

dale_chall_readability_score 0.4248152976966536 

difficult_words 0.7209908735332464 

linsear_write_formula 0.6573229030856149 

gunning_fog 0.5332464146023469 

fernandez_huerta 0.47044763146458063 

szigriszt_pazos 0.4695784441547154 

gutierrez_polini 0.4382877009995654 

crawford 0.5562798783137766 

gulpease_index 0.3357235984354628 

osman 0.4002607561929596 



# Feature extraction: TF-IDF

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

In [22]:
tfidf_model_original = TfidfVectorizer(
    ngram_range=(1, 4), min_df=0.0005, stop_words="english"
)

tfidf_model_original.fit(df["definitions"])

TfidfVectorizer(min_df=0.0005, ngram_range=(1, 4), stop_words='english')

In [23]:
tfidf_df_train_original = pd.DataFrame(
    tfidf_model_original.transform(df["definitions"]).todense()
)
tfidf_df_train_original.columns = sorted(tfidf_model_original.vocabulary_)

# validation
tfidf_df_valid_original = pd.DataFrame(
    tfidf_model_original.transform(val_df["definitions"]).todense()
)
tfidf_df_valid_original.columns = sorted(tfidf_model_original.vocabulary_)


In [24]:
tfidf_df_train_original

,000,000 dollars,000 dollars assessed,000 dollars assessed value,000 mutual,000 mutual funds,000 mutual funds exchange,000 small,000 small denominations,000 small denominations enhance,10,10 20,10 20 days,10 countries,10 countries deposited,10 countries deposited funds,10 gab,10 gab 10,10 gab 10 countries,100,100 accounts,100 accounts receivable,100 accounts receivable ar,100 basis,100 basis points,100 basis points target,100 bounded,100 bounded range,100 bounded range values,100 years,100 years owe,100 years owe existence,1092,1092 release,1092 release securities,1092 release securities exchange,120,120 days,120 days date,120 days date veterans,...,yield maturity measures,yield maturity measures rate,yield modification,yield modification takes,yield modification takes account,yield quoted,yields,yields covariances,yields covariances assets,yields covariances assets alsocorrelation,york,york city,york city largest,york city largest equities,york stock,york stock exchange,york stock exchange nyse,zero,zero debt,zero debt alternative,zero debt alternative levered,zero minus,zero minus tick,zero zero,zero zero minus,zero zero minus tick,σ2,σ2 r1,σ2 r1 variance,σ2 r1 variance return,σ2 r2,σ2 r2 variance,σ2 r2 variance return,σ2 variance,σ2 variance value,σ2 variance value d2,σ22σ,σ22σ ts,σ22σ ts stock,σ22σ ts stock price
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.042214,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.049391,0.049391,0.049391,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
398,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [25]:
tfidf_df_valid_original

,000,000 dollars,000 dollars assessed,000 dollars assessed value,000 mutual,000 mutual funds,000 mutual funds exchange,000 small,000 small denominations,000 small denominations enhance,10,10 20,10 20 days,10 countries,10 countries deposited,10 countries deposited funds,10 gab,10 gab 10,10 gab 10 countries,100,100 accounts,100 accounts receivable,100 accounts receivable ar,100 basis,100 basis points,100 basis points target,100 bounded,100 bounded range,100 bounded range values,100 years,100 years owe,100 years owe existence,1092,1092 release,1092 release securities,1092 release securities exchange,120,120 days,120 days date,120 days date veterans,...,yield maturity measures,yield maturity measures rate,yield modification,yield modification takes,yield modification takes account,yield quoted,yields,yields covariances,yields covariances assets,yields covariances assets alsocorrelation,york,york city,york city largest,york city largest equities,york stock,york stock exchange,york stock exchange nyse,zero,zero debt,zero debt alternative,zero debt alternative levered,zero minus,zero minus tick,zero zero,zero zero minus,zero zero minus tick,σ2,σ2 r1,σ2 r1 variance,σ2 r1 variance return,σ2 r2,σ2 r2 variance,σ2 r2 variance return,σ2 variance,σ2 variance value,σ2 variance value d2,σ22σ,σ22σ ts,σ22σ ts stock,σ22σ ts stock price
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.128957,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
94,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
95,0.0,0.0,0.0

# Logistic Regression Model

In [26]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score

In [27]:
train_X = tfidf_df_train_original
train_y = df['assigned_readability']
valid_X = tfidf_df_valid_original
valid_y = val_df['assigned_readability']


In [28]:
def model(clf, train_X, train_y, valid_X, valid_y):
    clf.fit(train_X, train_y)
    pred_tr = clf.predict(train_X)
    pred_valid = clf.predict(valid_X)
    pred_tr_prob = clf.predict_proba(train_X)[:,1]
    pred_valid_prob = clf.predict_proba(valid_X)[:,1]
    print("\nTraining F1:{}".format(f1_score(train_y, pred_tr, average="weighted")))
    print("Training Confusion Matrix \n{}".format(confusion_matrix(train_y, pred_tr)))
    print("Classification Report Train: \n{}".format(classification_report(train_y, pred_tr)))
    print("AUC Train", roc_auc_score(train_y, pred_tr_prob))

    print(
        "\nValidation F1:{}".format(f1_score(valid_y, pred_valid, average="weighted"))
    )
    print(
        "Validation Confusion Matrix \n{}".format(confusion_matrix(valid_y, pred_valid))
    )
    print(
        "Classification Report: \n{}".format(classification_report(valid_y, pred_valid))
    )
    print("AUC Valid", roc_auc_score(valid_y, pred_valid_prob))

In [29]:
lr_clf = LogisticRegression(solver="lbfgs", n_jobs=-1)
model(lr_clf, train_X, train_y, valid_X, valid_y)
params = lr_clf.get_params()
print(params)


Training F1:1.0
Training Confusion Matrix 
[[171   0]
 [  0 231]]
Classification Report Train: 
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       171
           1       1.00      1.00      1.00       231

    accuracy                           1.00       402
   macro avg       1.00      1.00      1.00       402
weighted avg       1.00      1.00      1.00       402

AUC Train 1.0

Validation F1:0.5383579788504549
Validation Confusion Matrix 
[[ 4 35]
 [ 0 59]]
Classification Report: 
              precision    recall  f1-score   support

           0       1.00      0.10      0.19        39
           1       0.63      1.00      0.77        59

    accuracy                           0.64        98
   macro avg       0.81      0.55      0.48        98
weighted avg       0.78      0.64      0.54        98

AUC Valid 0.7548891786179922
{'C': 1.0, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_r